<a href="https://colab.research.google.com/github/Huwalli/PLta/blob/main/%E6%96%87%E5%AD%97%E5%88%86%E9%A1%9E%E6%96%B7%E5%AD%97_Part2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**大家為什麼會來上這門課**
https://github.com/peculab/PythonAI4Beginners/issues/1

In [ ]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

In [ ]:
import pandas as pd
# read data and put it in a dataframe
# 在 google 工作表載入 gsheets
gsheets = gc.open_by_url('https://docs.google.com/spreadsheets/d/14enbqugRpENropj6T_S3SWldovCtosMuamCUZev42HI/edit?usp=sharing').sheet1

In [ ]:
dicts = gc.open_by_url('https://docs.google.com/spreadsheets/d/14enbqugRpENropj6T_S3SWldovCtosMuamCUZev42HI/edit?usp=sharing').get_worksheet(1)
dicts = dicts.get_all_records()
dicts = pd.DataFrame(dicts)

In [ ]:
# 讀取所有數據
rows = gsheets.get_all_records()
import pandas as pd
df = pd.DataFrame(rows)

In [ ]:
df.head()

,Account,Date,Cleaned_Content
0,peculab,"Aug 27, 2024",Owner peculab • 學校、姓名、以及你為什麼來上這門課？並留下你的第一個 co...
1,pupupeter,"Sep 2, 2024",Collaborator pupupeter • 高中生們兒早哇，我是這次的助教，我叫家瑋...
2,Junxian0829,"Sep 2, 2024",Collaborator Junxian0829 • 各位好，我是鄭鈞賢，是台師大機電系大...
3,tedlu-tw,"Sep 2, 2024",Collaborator tedlu-tw 各位好，和大家一樣是高中生，就讀北市數位實中，...
4,Eric970308-123,"Sep 2, 2024",Eric970308-123 大家好，我是胡程勛，目前高二，就讀彰化縣溪湖高中207班，在...


從以上資料我們可以知道，這個資料會有：帳號、日期、角色、內容

In [ ]:
!pip install jieba
!pip install snownlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.6/37.6 MB 48.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for snownlp: filename=snownlp-0.12.3-py3-none-any.whl size=37760946 sha256=e60c4876be40c459e7244d5183523970dec774597b49b3e73b4d5ab9c6928035
  Stored in directory: /root/.cache/pip/wheels/4a/fc/04/d1937c02b2a445b34754da55f71612a3df648a38d711bd17eb
Successfully built snownlp


In [ ]:
import jieba
from snownlp import SnowNLP
import pandas as pd
from collections import Counter
import re
import plotly.express as px

In [ ]:
#將自訂詞加入 Jieba 的斷詞系統中，確保在斷詞時能夠正確地把這些詞視為單一詞彙。
for _, row in dicts.iterrows():
    word = row['words']
    jieba.add_word(word, freq=10000)
    jieba.suggest_freq(word, tune=True)

Building prefix dict from the default dictionary ...
DEBUG:jieba:Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
DEBUG:jieba:Dumping model to file cache /tmp/jieba.cache
Loading model cost 1.587 seconds.
DEBUG:jieba:Loading model cost 1.587 seconds.
Prefix dict has been built successfully.
DEBUG:jieba:Prefix dict has been built successfully.


In [ ]:
# 只保留中文字
def extract_chinese(text):
    return ''.join(re.findall(r'[\u4e00-\u9fa5]', text))

In [ ]:
df['Chinese_Content'] = df['Cleaned_Content'].apply(extract_chinese)

# 使用 Jieba 斷詞
df['Tokenized'] = df['Chinese_Content'].apply(lambda x: list(jieba.cut(x, HMM=True)))

# 展平成所有詞語的列表
all_words = [word for tokens in df['Tokenized'] for word in tokens if len(word) > 1]

In [ ]:
df['Tokenized']

,Tokenized
0,"[學校, 姓名, 以及, 你, 為什麼, 來上, 這門課, 並, 留下, 你, 的, 第一,..."
1,"[高中生, 們, 兒, 早, 哇, 我, 是, 這次, 的, 助教, 我, 叫, 家瑋, 現..."
2,"[各位, 好, 我, 是, 鄭鈞賢, 是, 台師大機電系, 大三, 的, 從, 前, 我, ..."
3,"[各位, 好, 和, 大家, 一樣, 是, 高中生, 就, 讀北市, 數位, 實中, 同時,..."
4,"[大家, 好, 我, 是, 胡程勛, 目前, 高二, 就, 讀, 彰化縣, 溪湖高中, 班,..."
...,...
307,"[我, 是, 嘉工, 陳信錩, 希望, 可以, 學到, 東西]"
308,"[我, 是, 南港高中, 高廷翰, 對, 程式, 基礎, 有, 幫助]"
309,"[我, 是, 鳳梨大學, 傳播系, 的, 漁人在網, 路上, 路過, 一起, 來學, 的, ..."
310,"[我, 是, 鳳梨大學, 傳播系, 的, 漁人在網, 路上, 路過, 一起, 來學, 的, ..."


這段我們沒有使用NER模組而是：手工規則式 NER

1. 學校名稱抽取（NER 的 Location / Organization 類）

2. 角色辨識（NER 的 Person / Title 類）

In [ ]:
import re  # 匯入正則表達式模組，用來進行文字模式比對
import pandas as pd  # 匯入 pandas 模組，用來處理資料表格

# 定義助教的帳號名單，用來判斷使用者是否為助教身份
assistants = ['pupupeter', 'Junxian0829', 'tedlu-tw']
teacher = ['peculab']

# 定義角色判定規則的函式，根據帳號和內容判斷是助教、老師、學生、大學生或未知
def determine_role(row):
    if row['Account'] in assistants:
        return '助教'  # 如果帳號在助教名單中，標記為助教
    elif row['Account'] in teacher:
        return '老師'  # 如果帳號在老師名單中，標記為老師
    elif '助教' in row['Cleaned_Content'] or 'Collaborator' in row['Cleaned_Content']:
        return '助教'  # 如果內容中出現「助教」或「Collaborator」，也標記為助教
    elif any(keyword in row['Cleaned_Content'] for keyword in ['老師', '數學老師']):
        return '老師'  # 如果內容中出現「老師」或「數學老師」，標記為老師
    elif re.search(r'(高中|附中|高工|高商|工商)', row['Cleaned_Content']):
        return '學生'  # 如果出現高中、附中、高工、高商、工商，推定是高中職學生
    elif re.search(r'(大學|台師大)', row['Cleaned_Content']):
        return '大學生'  # 如果出現大學、台師大等詞彙，推定是大學生
    else:
        return '未知'  # 無法判定時，標記為未知

# 定義學校名稱提取函式，從文字內容中找出提到的學校名稱
def extract_school(content):
    # 特例處理：遇到特定學校名稱時直接返回
    if '數位實中' in content:
        return '數位實中'
    if '台師大' in content or '臺師大' in content or '師範大學' in content:
        return '臺師大'

    # 用正則表達式比對常見的學校結尾詞彙
    school_keywords = r'(高中|附中|高工|高商|工商|大學|中學|國中|國小)'
    # 嘗試從文字中找出類似「臺北高中」、「成功中學」等學校名稱
    match = re.search(r'([\u4e00-\u9fa5]+?' + school_keywords + ')', content)
    if match:
        school = match.group(0)  # 取得符合的學校名稱片段
        return school[-4:]  # 回傳最後四個字，通常是學校的完整名稱
    return None  # 若未匹配到學校名稱則回傳 None

# 將 extract_school 函數應用到 Cleaned_Content 欄位，新增一欄 School 學校名稱
df['School'] = df['Cleaned_Content'].apply(lambda x: extract_school(str(x)))

# 使用 determine_role 函式，判斷每筆資料的角色身份，新增一欄 Role
df['Role'] = df.apply(determine_role, axis=1)




In [ ]:
df['Role']

,Role
0,未知
1,助教
2,助教
3,助教
4,學生
...,...
307,未知
308,學生
309,大學生
310,助教


In [ ]:
df['School']

,School
0,None
1,臺師大
2,臺師大
3,數位實中
4,溪湖高中
...,...
307,None
308,南港高中
309,鳳梨大學
310,鳳梨大學


In [ ]:
df.head()

,Account,Date,Cleaned_Content,Chinese_Content,Tokenized,School,Role
0,peculab,"Aug 27, 2024",Owner peculab • 學校、姓名、以及你為什麼來上這門課？並留下你的第一個 co...,學校姓名以及你為什麼來上這門課並留下你的第一個程式連結的我想要透過寫程式來實現我心中的想法我...,"[學校, 姓名, 以及, 你, 為什麼, 來上, 這門課, 並, 留下, 你, 的, 第一,...",None,老師
1,pupupeter,"Sep 2, 2024",Collaborator pupupeter • 高中生們兒早哇，我是這次的助教，我叫家瑋...,高中生們兒早哇我是這次的助教我叫家瑋現為台灣師範大學歷史系三年級的同學也是個愛大便的小孩子這...,"[高中生, 們, 兒, 早, 哇, 我, 是, 這次, 的, 助教, 我, 叫, 家瑋, 現...",臺師大,助教
2,Junxian0829,"Sep 2, 2024",Collaborator Junxian0829 • 各位好，我是鄭鈞賢，是台師大機電系大...,各位好我是鄭鈞賢是台師大機電系大三的從前我的夢想是成為一名頂尖的歌手可惜因為一些個人因素我唱...,"[各位, 好, 我, 是, 鄭鈞賢, 是, 台師大機電系, 大三, 的, 從, 前, 我, ...",臺師大,助教
3,tedlu-tw,"Sep 2, 2024",Collaborator tedlu-tw 各位好，和大家一樣是高中生，就讀北市數位實中，...,各位好和大家一樣是高中生就讀北市數位實中同時在臺大資工擔任研究助理雖然如此但我總是宣稱自己是...,"[各位, 好, 和, 大家, 一樣, 是, 高中生, 就, 讀北市, 數位, 實中, 同時,...",數位實中,助教
4,Eric970308-123,"Sep 2, 2024",Eric970308-123 大家好，我是胡程勛，目前高二，就讀彰化縣溪湖高中207班，在...,大家好我是胡程勛目前高二就讀彰化縣溪湖高中班在班上擔任的幹部是設備股長下面的程式是我在高一的...,"[大家, 好, 我, 是, 胡程勛, 目前, 高二, 就, 讀, 彰化縣, 溪湖高中, 班,...",溪湖高中,學生


In [ ]:
#統計每個學校出現的次數
from collections import Counter

school_counts = Counter(df['School'])

In [ ]:
school_counts

Counter({None: 39,
         '臺師大': 2,
         '數位實中': 2,
         '溪湖高中': 13,
         '中山附中': 25,
         '中山大學': 4,
         '南投高中': 2,
         '羅東高工': 14,
         '鼓山高中': 33,
         '仁武高中': 12,
         '關山工商': 2,
         '東海國中': 1,
         '中山高中': 9,
         '花蓮高工': 7,
         '楠梓高中': 17,
         '中港高中': 6,
         '清水高中': 31,
         '新社高中': 2,
         '林園高中': 15,
         '基隆高中': 30,
         '嘉義高工': 13,
         '南港高中': 19,
         '新竹高商': 10,
         '竹北高中': 2,
         '鳳梨大學': 2})

In [ ]:
import plotly.express as px  # 匯入 plotly.express，用來畫互動式圖表

# 將 school_counts（Counter 物件）依照數量由大到小排序，得到一個 list of tuples
sorted_school_counts = sorted(school_counts.items(), key=lambda x: x[1], reverse=True)

# 將排序後的 list 轉成 dict，方便轉成 DataFrame
sorted_school_counts_dict = dict(sorted_school_counts)

# 建立一個 DataFrame，欄位分別是學校名稱（School）與出現次數（Count）
sorted_school_df = pd.DataFrame(list(sorted_school_counts_dict.items()), columns=["School", "Count"])

# 繪製長條圖：x 軸是學校名稱，y 軸是提及次數
fig_sorted = px.bar(sorted_school_df, x='School', y='Count',
                    title="School Distribution (Sorted by Count)",  # 圖表標題
                    labels={"School": "School", "Count": "Number of Mentions"},  # x, y 軸的標籤
                    text='Count')  # 在每個柱子上標出數值

# 設定數字顯示在柱狀圖外側
fig_sorted.update_traces(texttemplate='%{text}', textposition='outside')

# 調整 x 軸與 y 軸的標題
fig_sorted.update_layout(xaxis=dict(title="School"), yaxis=dict(title="Number of Mentions"))

# 顯示圖表
fig_sorted.show()


算完學校部分，我們還可以看每個帳號的留言次數

In [ ]:
# 讓account_message_count保留 Account-Message_Count 配對，計算留言次數
account_message_count = df.groupby('Account').size().reset_index(name='Message_Count')

# 讓account_school保留唯一的 Account-School 配對
account_school = df[['Account', 'School']].drop_duplicates()

# 合併留言次數與學校名稱
result = account_message_count.merge(account_school, on='Account')

In [ ]:
result

,Account,Message_Count,School
0,40112,1,鼓山高中
1,40114,1,鼓山高中
2,210136,1,嘉義高工
3,218111,1,羅東高工
4,311257,1,清水高中
...,...,...,...
278,zhen910,1,花蓮高工
279,zhenjane,1,楠梓高中
280,zhenxinglin21024,1,基隆高中
281,zquonn,1,仁武高中


In [ ]:
filtered_result = result[result['Message_Count'] > 1]
filtered_result

,Account,Message_Count,School
18,131001yt2019snipe,2,鼓山高中
19,19-bryce,2,中山附中
24,Alan960609,2,花蓮高工
55,Irving-1223,2,林園高中
61,Jiaxuan210357,2,溪湖高中
64,Jusus62,2,羅東高工
83,Nily210251,2,溪湖高中
93,Rex0511,2,中山附中
100,Shinlinwu,2,鼓山高中
126,ZhongYi-Su,2,溪湖高中


In [ ]:
result.describe()

,Message_Count
count,283.000000
mean,1.385159
std,1.565381
min,1.000000
25%,1.000000
50%,1.000000
75%,1.000000
max,11.000000


來看哪間學校有最多留言、有哪些帳號來自該校

In [ ]:
# 根據 'School' 欄位進行分組，並進行彙總統計
grouped_data = result.groupby('School').agg({
    # 將相同學校的所有帳號合併成一個字串（帳號之間用逗號分隔）
    'Account': lambda accounts: ', '.join(map(str, accounts)),  # 先轉成字串再合併
    # 將相同學校的留言總數加總
    'Message_Count': 'sum'
}).reset_index()  # 重設索引，讓 'School' 成為一個欄位而不是索引

# 重新命名欄位讓結果更易讀
grouped_data.rename(columns={
    'School': 'School_Name',         # 把原本的 'School' 改成 'School_Name'
    'Account': 'Accounts',           # 把合併後的帳號欄位命名為 'Accounts'
    'Message_Count': 'Total_Messages' # 把總留言數欄位命名為 'Total_Messages'
}, inplace=True)

# 根據總留言數由大到小排序，並重設索引
sorted_data = grouped_data.sort_values(by='Total_Messages', ascending=False).reset_index(drop=True)

# 顯示最終排序後的資料
sorted_data


,School_Name,Accounts,Total_Messages
0,基隆高中,"21018Hank, An4567ny, Eeeeeeeemma, Kien0516, Le...",40
1,中山附中,"19-bryce, Anson594412, DingSianWu, I2amgay, Ia...",35
2,鼓山高中,"40112, 40114, 131001yt2019snipe, Andy980325, B...",33
3,清水高中,"311257, 311259, 311264, 311272, 10802-a, Allen...",31
4,楠梓高中,"971010ian, CaiBingyan123, Pei-0325, Rebecca-10...",27
5,羅東高工,"218111, 218234nick, JEFF-GMT, Jeff-1024, Jusus...",24
6,南港高中,"05yc, 1024Amon, 10909Jasmine, 10921Hennessy, 1...",19
7,溪湖高中,"Eric970308-123, Jiaxuan210357, Lin434255, Nily...",15
8,林園高中,"000000-Nora, Albert980206, Chuxin014, Irving-1...",15
9,中山大學,"Rna0925, kai081024, peculab",14


看每間學校的帳號有哪些，以及調閱他們都說了什麼話

In [ ]:
raw_accounts = list(sorted_data[sorted_data['School_Name'] == '東海國中']['Accounts'])
raw_accounts

['QiuDaru']

In [ ]:
# 將帳號字串用逗號分隔並轉為集合，方便後續進行篩選與比對（集合中的值不會重複，查找速度也比較快）
accounts_to_extract = set(raw_accounts[0].split(', '))
accounts_to_extract

{'QiuDaru'}

In [ ]:
# 篩選出帳號在 accounts_to_extract 裡的資料
filtered_df = df[df['Account'].isin(accounts_to_extract)]
filtered_df

,Account,Date,Cleaned_Content,Chinese_Content,Tokenized,School,Role
110,QiuDaru,"Sep 3, 2024",QiuDaru 大家好，我是東海國中邱達儒，因為想要讓AI幫我寫作業XD ( 開完笑的 )...,大家好我是東海國中邱達儒因為想要讓幫我寫作業開完笑的這感覺很帥所以我想學,"[大家, 好, 我, 是, 東海國, 中, 邱達儒, 因為, 想要, 讓, 幫, 我, 寫,...",東海國中,未知


In [ ]:
from google.colab import userdata
apikey = userdata.get('test')

In [ ]:
import google.generativeai as genai
genai.configure(api_key = apikey)
model = genai.GenerativeModel("gemini-1.5-flash")

prompt = f"用中文分析社交媒體上對特定事件的風向：{filtered_df.to_string()}"

response = model.generate_content(prompt)
print(response.text)

這條社交媒體數據顯示東海國中學生邱達儒在2024年9月3日發布了一則關於使用AI寫作業的帖子。

**風向分析：**

從提供的數據來看，單純依靠這一個帖子難以判斷整體社交媒體的風向。  邱達儒的帖子語氣輕鬆，帶有玩笑成分("開完笑的")，表達了對AI寫作業的好奇和學習意願。  他分享了一個Google Colab連結，暗示其目的是學習如何利用AI輔助學習。

**缺乏數據導致的局限性：**

* **樣本量不足：**僅憑一條帖子無法推斷出大眾對AI寫作業的看法。  需要更多數據才能分析輿論的整體傾向。
* **缺乏互動數據：**  沒有顯示其他用戶的回覆和評論，無法了解帖子引發的反響。 這些回覆可能表達了支持、反對、或中立的觀點，才能更全面地了解風向。
* **情緒分析缺失：**  雖然帖子語氣輕鬆，但缺乏更精細的情緒分析工具，無法準確判斷其情感傾向（例如：是純粹的好奇，還是對學業壓力的反映）。

**要了解社交媒體對AI寫作業的風向，需要收集更多數據，包括：**

* **更多帖子：**  從不同用戶的發文，觀察不同觀點的比例。
* **評論和回覆：**  分析用戶對邱達儒帖子的反應，以及其他相關帖子的討論。
* **情緒分析：**  使用自然語言處理技術，對文本進行情感分析，判斷正負面情緒的比例。
* **話題熱度：**  觀察相關話題的討論量和搜索量，了解其關注度。


總而言之，目前僅從一條帖子無法判斷社交媒體對AI寫作業的風向。  需要更大量的數據和更全面的分析才能得出可靠的結論。

